# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Adding the CSV
weather_data_path = "Output_data/cities.csv"
# Reading the CSV
weather_csv = pd.read_csv(weather_data_path)
weather_csv.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tasiilaq,65.6145,-37.6368,33.96,98,100,14.63,GL,1650565490
1,1,Albany,42.6001,-73.9662,58.03,44,99,12.95,US,1650565388
2,2,Tauá,-5.9667,-40.3000,83.64,57,100,8.43,BR,1650565927
3,3,Gravelbourg,49.8834,-106.5512,35.87,57,48,14.63,CA,1650565603
4,4,Potamiá,40.7163,24.7286,54.64,74,61,2.19,GR,1650565928


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humid = weather_csv["Humidity"].astype(float)
max_humid = humid.max()
print(f' The Maximum Humidity Level is {max_humid}')

 The Maximum Humidity Level is 100.0


In [4]:
# Configure Gmaps
city_location = weather_csv[["Lat", "Lng"]].astype(float)
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

# Creating the Heat Layer
heat_layer = gmaps.heatmap_layer(city_location, weights=humid, max_intensity=100, dissipating=False, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

In [5]:
# Creating the new dataframe 
new_city_df = weather_csv.loc[(weather_csv["Max Temp"] > 70) & (weather_csv["Max Temp"] < 80) & (weather_csv["Wind Speed"] < 10) & (weather_csv["Cloudiness"] == 0)]
new_city_df = new_city_df.dropna(how='any')

new_city_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
187,187,Concepción del Uruguay,-32.4825,-58.2372,73.98,58,0,1.99,AR,1650565994
194,194,Lima,-12.0432,-77.0282,71.85,64,0,6.91,PE,1650565826
253,253,Umluj,25.0213,37.2685,76.37,66,0,9.69,SA,1650566018
288,288,Hīt,33.6416,42.8251,72.28,20,0,9.78,IQ,1650566030
319,319,Golden,39.7555,-105.2211,74.25,30,0,8.99,US,1650565812
482,482,Valparaíso,-33.0393,-71.6273,73.63,58,0,9.22,CL,1650565555
530,530,Lakki,32.6079,70.9114,74.95,43,0,5.88,PK,1650566136


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
output_data_file = "../output_data/vacation_cities.csv"
new_city_df.to_csv(output_data_file )

In [7]:
# Storing info into new hotel df
hotel_df = pd.read_csv("../output_data/vacation_cities.csv")
# adding new column
hotel_df['Hotel Name'] = ''

hotel_df

,Unnamed: 0,Unnamed: 0.1,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,187,187,Concepción del Uruguay,-32.4825,-58.2372,73.98,58,0,1.99,AR,1650565994,
1,194,194,Lima,-12.0432,-77.0282,71.85,64,0,6.91,PE,1650565826,
2,253,253,Umluj,25.0213,37.2685,76.37,66,0,9.69,SA,1650566018,
3,288,288,Hīt,33.6416,42.8251,72.28,20,0,9.78,IQ,1650566030,
4,319,319,Golden,39.7555,-105.2211,74.25,30,0,8.99,US,1650565812,
5,482,482,Valparaíso,-33.0393,-71.6273,73.63,58,0,9.22,CL,1650565555,
6,530,530,Lakki,32.6079,70.9114,74.95,43,0,5.88,PK,1650566136,


In [8]:
# Set parameters to search for hotels with 5000 meters
# params = {
#     "radius": 5000,
#     "types": "hotel",
#     "key": g_key
# }
hotels = []

for city in range(len(hotel_df["City"])):
    lat = hotel_df.loc[city]['Lat']
    lng = hotel_df.loc[city]['Lng']
    
    city_locat = f"{lat},{lng}"
    
    params = {"location": city_locat,
             "types": "lodging",
             "radius": 5000,
             "key": g_key}
    
    params['location'] = f"{lat},{lng}"
    
    b_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_plot = requests.get(b_url, params=params)
    hotel_result = hotel_plot.json()
    try:
        hotels.append(hotel_result["results"][0]["name"])
        
    except (KeyError, IndexError):
            print("Missing....Skipping")
            
hotel_df["Hotel Name"] = hotels
hotel_df

,Unnamed: 0,Unnamed: 0.1,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,187,187,Concepción del Uruguay,-32.4825,-58.2372,73.98,58,0,1.99,AR,1650565994,Aires del Campo Cabañas y Hotel
1,194,194,Lima,-12.0432,-77.0282,71.85,64,0,6.91,PE,1650565826,Hotel Kamana
2,253,253,Umluj,25.0213,37.2685,76.37,66,0,9.69,SA,1650566018,Moon light Furnished Units
3,288,288,Hīt,33.6416,42.8251,72.28,20,0,9.78,IQ,1650566030,بيت ابو رباح
4,319,319,Golden,39.7555,-105.2211,74.25,30,0,8.99,US,1650565812,"The Golden Hotel, Ascend Hotel Collection"
5,482,482,Valparaíso,-33.0393,-71.6273,73.63,58,0,9.22,CL,1650565555,Hotel Ultramar
6,530,530,Lakki,32.6079,70.9114,74.95,43,0,5.88,PK,1650566136,Noor Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, fill_color="rgba(0,150,0,0.4)", stroke_color="rgba(0,0,150,0.4)", 
                                 scale=3, info_box_content=hotel_info)
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))